# pyxem


### A 4-D STEM Package in the Hyperspy Ecosystem

#### Carter Francis, University of Wisconsin Madison
#### *May 16, 2024*

In [ ]:
import pyxem as pxm
import hyperspy.api as hs

### Download the Data (Try to do this ahead of time!)

In [ ]:
pxm.data.mgo_nanocrystals(allow_download=True)
pxm.data.zrnb_precipitate(allow_download=True)
pxm.data.twinned_nanowire(allow_download=True)

# Outline

1. Introduction to 4D STEM
2. Introduction to pyxem
3. Example Workflows
    1. Loading and Visualizing Data (This Notebook)
    2. Creating Virtual Images (MgO Nanocrystals) (This Notebook)
    3. Strain Mapping and Finding Vectors (Inclusions in ZrNb) (30 min) 
    4. Azimuthal Integration And Orientation Mapping in Molecular Glasses (30 min)

# Introduction to 4D STEM

<img src="Images/4DSTEM.gif" alt="4D STEM GIF">

# Introduction to pyxem (pix-em)


Because pyxem extends hyperspy importing hyperspy will automatically load pyxem if it is installed.

This also means that we inherit all of the upstream hyperspy functionality such as:

1. Fast I-O (from [many different file formats](https://hyperspy.org/rosettasciio/supported_formats/index.html))
2. Machine learning with [Matrix Factorization](https://hyperspy.org/hyperspy-doc/current/user_guide/mva.html)
3. Fitting with [1-D and 2-D functions](https://hyperspy.org/hyperspy-doc/current/user_guide/model.html)
4. [Interactive Plotting](https://hyperspy.org/hyperspy-doc/current/user_guide/visualisation.html)
5. [Metadata Handling](https://hyperspy.org/hyperspy-doc/current/user_guide/metadata_structure.html)
6. And __Much Much__ More!

For more information about pyxem check out our [github](https://github.com/pyxem/pyxem) our [documentation](https://pyxem.readthedocs.io/en/latest/) or the set of [tutorial notebooks](https://github.com/pyxem/pyxem-demos) or our new set of [examples](https://pyxem.readthedocs.io/en/stable/examples/index.html)

## Loading and Visualizing Data

Data in `pyxem` can be loaded exactly the same as loading data in `hyperspy`.  Many different formats are supported but only the `.hspy` (hdf5) and `.zspy` (zarr) formats retain all of the metadata

```python
import hyperspy.api as hs
s = hs.load("path/to/file/to/load.hspy")
```

Visualization can easily be done by then calling the function

```python
s.plot()
```


In [ ]:
# Starting up a distributed Cluster locally 
# You don't have to do this but it helps to visualize what is happening. Note that you can 
from dask.distributed import Client
client = Client()  # set up local cluster on your laptop
client
# Ignore INFO below just giving informataion about the scheduler set up

In [ ]:
client

In [ ]:
import pyxem as pxm
import hyperspy.api as hs
hs.set_log_level("ERROR")

In [ ]:
# setting the plotting backend (note this works much better with the qt backend!)
%matplotlib qt
dp = pxm.data.mgo_nanocrystals(lazy=True)

## Creating Virtual Images
We can construct Virtual images very easily using [ROIS](http://hyperspy.org/hyperspy-doc/current/user_guide/interactive_operations_ROIs.html) from `hyperspy`.

In [ ]:
import hyperspy
print(hyperspy.__version__)

In [ ]:
# This will work with lazy signals if they are small and stored efficiently.
dp.compute() 

In [ ]:
# plot the data
dp.plot()

In [ ]:
# Lets make an couple of interactive ROI's 
hs.plot.plot_roi_map(dp, rois=3,single_figure=True)

We can also define a specific ROI to create virtual Images.

In [ ]:
mean_cbed = dp.mean() # get the mean diffraction pattern
mean_cbed.plot(vmax=1000) # plot the mean diffraction pattern
df_roi = hs.roi.CircleROI(cx=0, cy=0, r= 2.0, r_inner=0.5) # create a ROI to be used as a virtual appature
df_roi.add_widget(mean_cbed) # add roi to plot
roi = hs.roi.CircleROI(cx=0.12984, cy=1.4607, r=0.24345, r_inner=0) # create a ROI to be used as a virtual appature
roi.add_widget(mean_cbed) # add roi to plot

In [ ]:
# get the integrated intensity to make the virtual images.
df =  dp.get_integrated_intensity(df_roi) # integrate the DF ROI 
virtual =  dp.get_integrated_intensity(roi) # integrate the DF ROI 

hs.plot.plot_images([df, virtual],
                    label=["Virtual Dark Field", "Virtual Aperture"],
                    tight_layout=True,
                    colorbar=None, 
                    scalebar="all", 
                    axes_decor="off") # Plot both ROI's using hyperspy

### Finding Diffraction Vectors and Crystal Segmentation

For many operations in `pyxem` such as Crystal Segmentation, Strain Mapping and Clustering we start with finding the set of diffraction vectors within the dataset.  This starts with the function `s.get_diffraction_vectors()`

##### Removing Background

For more infromation look at the [background subtraction example](https://pyxem.readthedocs.io/en/stable/examples/processing/background_subtraction.html#sphx-glr-examples-processing-background-subtraction-py)

In [ ]:
import pyxem as pxm
import hyperspy.api as hs
hs.set_log_level("ERROR")

In [ ]:
dp = pxm.data.mgo_nanocrystals(lazy=True)

In [ ]:
# find peaks using difference of gaussians 
dv = dp.get_diffraction_vectors(method="difference_of_gaussian",
                                min_sigma=2, max_sigma=4,
                                threshold=.02)

In [ ]:
%matplotlib ipympl
#plot the markers over the dataset
m = dv.to_markers(facecolor="none", edgecolor="w", sizes=20)
dp.plot()
dp.add_marker(m)

In [ ]:
# compute the diffraction vectors
dv.compute()

In [ ]:
# plot the map (number found) of diffraction vectors 
dv.get_diffracting_pixels_map().plot()

In [ ]:
import numpy as np
distance_threshold = 0.1
min_samples = 7
# cluster the vectors to get only the unique vectors
unique_peaks = dv.get_unique_vectors(method='DBSCAN',
                                          distance_threshold=distance_threshold,
                                          min_samples=min_samples, columns = (-3,-2))
print(np.shape(unique_peaks.data)[0], ' unique vectors were found.')

#remove the zero beam
unique_peaks = unique_peaks.filter_magnitude(min_magnitude=.4,
                                   max_magnitude=np.inf) 
print(np.shape(unique_peaks)[0], ' unique vectors.')

### Creating Virtual Images from Vectors

In [ ]:
rois, texts = unique_peaks.to_roi(radius=.1,
                                  include_labels=True,
                                  sizes=2,
                                  facecolor="black", color="w")
mean_dp = dp.mean()
mean_dp.plot(vmax="99th")
vdfs = dp.get_virtual_image(rois)
mean_dp.add_marker(texts)


In [ ]:
# display the vdfs object
vdfs

In [ ]:
# plot the vdfs (lazily)
vdfs.plot()

In [ ]:
# compute VDFS
vdfs.compute()

### Creating Segmented Detectors

In [ ]:
import pyxem as pxm
from pyxem.utils._azimuthal_integrations import _get_control_points
import numpy as np
import hyperspy.api as hs

dp.calibration.center = None  # Center the diffraction patterns


# For Visualizing the virtual detector
cp = _get_control_points(
    1,
    npt_azim=8,
    radial_range=(1, 2),
    azimuthal_range=(-np.pi, np.pi),
    affine=dp.calibration.affine,
)[:, :, ::-1]
poly = hs.plot.markers.Polygons(verts=cp, edgecolor="w", facecolor="none")
dp.plot()
dp.add_marker(poly)
pos = np.mean(cp, axis=1)
texts = np.arange(len(pos)).astype(str)
texts = hs.plot.markers.Texts(offsets=pos, texts=texts, color="w")
dp.add_marker(texts)

In [ ]:
# Create a 2D Azimuthal Integral and plot the transpose
az2d = dp.get_azimuthal_integral2d(npt=1, npt_azim=8, radial_range=(2, 5))
az2d.compute()
az2d.T.plot()

In [ ]:
# Createa 1D Azimuthal Integral and plot the transpose
az1d = dp.get_azimuthal_integral1d(npt=100)
az1d.compute()
az1d.T.plot()